In [8]:
!pip install synapseml

StatementMeta(, 8b7e04f3-d0a7-4d41-854c-ccc0ae3c391a, 10, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 574.6/574.6 kB 3.1 MB/s eta 0:00:00a 0:00:01


In [4]:
import pandas as pd
 
# Load data from the Lakehouse
df = pd.read_csv("/lakehouse/default/Files/predictive_maintenance.csv")
print(df)

StatementMeta(, 8b7e04f3-d0a7-4d41-854c-ccc0ae3c391a, 6, Finished, Available, Finished)

        UDI Product ID Type  Airtemperature  Processtemperature  \
0         1     M14860    M           298.1               308.6   
1         2     L47181    L           298.2               308.7   
2         3     L47182    L           298.1               308.5   
3         4     L47183    L           298.2               308.6   
4         5     L47184    L           298.2               308.7   
...     ...        ...  ...             ...                 ...   
9995   9996     M24855    M           298.8               308.4   
9996   9997     H39410    H           298.9               308.4   
9997   9998     M24857    M           299.0               308.6   
9998   9999     H39412    H           299.0               308.7   
9999  10000     M24859    M           299.0               308.7   

      Rotationalspeed  Torque  Toolwear  Target FailureType  
0                1551    42.8         0       0  No Failure  
1                1408    46.3         3       0  No Failure  
2        

In [5]:
from sklearn.preprocessing import OrdinalEncoder
 
# Check the unique values in the 'Type' column before encoding
print("Unique values in 'Type' before encoding:", df['Type'].unique())
 
# Encode 'Type' column using OrdinalEncoder
type_encoder = OrdinalEncoder(categories=[['L', 'M', 'H']])
df['Type'] = type_encoder.fit_transform(df[['Type']].values).astype(int)
 
# Encode 'Failure Type' column
failure_type_mapping = {
    'No Failure': 0,
    'Heat Dissipation Failure': 1,
    'Power Failure': 2,
    'Overstrain Failure': 3,
    'Tool Wear Failure': 4,
    'Random Failures': 5
}
df['FailureType'] = df['FailureType'].map(failure_type_mapping)
 

feature_columns = [
    'Type', 'Airtemperature', 'Processtemperature',
    'Rotationalspeed', 'Torque', 'Toolwear'
]
 
# Define target column
target_column = 'FailureType'
 
# Split data into features and target
X = df[feature_columns]
y = df[target_column]
 
print("Data prepared successfully.")

StatementMeta(, 8b7e04f3-d0a7-4d41-854c-ccc0ae3c391a, 7, Finished, Available, Finished)

Unique values in 'Type' before encoding: ['M' 'L' 'H']
Data prepared successfully.


In [6]:
from flaml import AutoML
 
# Initialize AutoML
automl = AutoML()
 
# Set AutoML configuration
automl_settings = {
    "time_budget": 60,  # Time budget in seconds
    "task": "classification",
    "metric": "accuracy",
    "estimator_list": ["lgbm", "rf", "xgboost"],  # List of estimators to consider
    "log_file_name": "automl_log.log",
    "log_type": "all"
}

StatementMeta(, 8b7e04f3-d0a7-4d41-854c-ccc0ae3c391a, 8, Finished, Available, Finished)

In [9]:
# Train the model
automl.fit(X_train=X, y_train=y, **automl_settings)
# Retrieve the best model
best_model = automl.model

StatementMeta(, 8b7e04f3-d0a7-4d41-854c-ccc0ae3c391a, 11, Finished, Available, Finished)

[flaml.automl.logger: 11-12 07:21:30] {1785} INFO - task = classification
[flaml.automl.logger: 11-12 07:21:30] {1796} INFO - Evaluation method: cv
class 5 augmented from 18 to 36
[flaml.automl.logger: 11-12 07:21:30] {1895} INFO - Minimizing error metric: 1-accuracy
[flaml.automl.logger: 11-12 07:21:30] {2013} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost']
[flaml.automl.logger: 11-12 07:21:30] {2323} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 11-12 07:21:30] {2458} INFO - Estimated sufficient time budget=1415s. Estimated necessary time budget=1s.


/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/11/12 07:21:43 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2024-04-16; if these packages are not present in the public PyPI index

[flaml.automl.logger: 11-12 07:21:52] {2507} INFO -  at 0.7s,	estimator lgbm's best error=0.0358,	best estimator lgbm's best error=0.0358
[flaml.automl.logger: 11-12 07:21:52] {2323} INFO - iteration 1, current learner lgbm


2024/11/12 07:22:05 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2024-04-16; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'synapseml-internal', 'synapseml-lightgbm', 'synapseml-deep-learning', 'synapseml-core', 'synapseml-vw', 'synapseml-mlflow', 'synapseml-opencv'}
2024/11/12 07:22:13 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2024-04-16; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'synapseml-internal', 'synapseml-lightgbm', 'synapseml-deep-learning', 'synapseml-core', 'synapseml-vw', 'synapseml-mlflow', 'synapseml-opencv'}


[flaml.automl.logger: 11-12 07:22:15] {2507} INFO -  at 23.0s,	estimator lgbm's best error=0.0358,	best estimator lgbm's best error=0.0358
[flaml.automl.logger: 11-12 07:22:15] {2323} INFO - iteration 2, current learner lgbm


2024/11/12 07:22:27 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2024-04-16; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'synapseml-internal', 'synapseml-lightgbm', 'synapseml-deep-learning', 'synapseml-core', 'synapseml-vw', 'synapseml-mlflow', 'synapseml-opencv'}
2024/11/12 07:22:35 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2024-04-16; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'synapseml-internal', 'synapseml-lightgbm', 'synapseml-deep-learning', 'synapseml-core', 'synapseml-vw', 'synapseml-mlflow', 'synapseml-opencv'}


[flaml.automl.logger: 11-12 07:22:37] {2507} INFO -  at 45.4s,	estimator lgbm's best error=0.0271,	best estimator lgbm's best error=0.0271
[flaml.automl.logger: 11-12 07:22:37] {2750} INFO - retrain lgbm for 0.0s
[flaml.automl.logger: 11-12 07:22:37] {2753} INFO - retrained model: LGBMClassifier(learning_rate=0.26770501231052046, max_bin=127,
               min_child_samples=12, n_estimators=1, n_jobs=-1, num_leaves=4,
               reg_alpha=0.001348364934537134, reg_lambda=1.4442580148221913,
               verbose=-1)
[flaml.automl.logger: 11-12 07:22:37] {2754} INFO - Auto Feature Engineering pipeline: None
[flaml.automl.logger: 11-12 07:22:37] {2756} INFO - Best MLflow run name: 
[flaml.automl.logger: 11-12 07:22:37] {2757} INFO - Best MLflow run id: 2363c9b3-df39-48ed-a06a-6df05a6a1381
[flaml.automl.logger: 11-12 07:22:37] {2049} INFO - fit succeeded
[flaml.automl.logger: 11-12 07:22:37] {2050} INFO - Time taken to find the best model: 45.36079239845276


In [10]:
model_type = type(best_model)
print(f"Best model type: {model_type}")
print("best hypermeter config:",automl.best_config)
print("best roc_auc: {0:.4g}".format(1- automl.best_loss))
print("training duration:{0:.4g}s".format(automl.best_config_train_time))

StatementMeta(, 8b7e04f3-d0a7-4d41-854c-ccc0ae3c391a, 12, Finished, Available, Finished)

Best model type: <class 'flaml.automl.model.LGBMEstimator'>
best hypermeter config: {'n_estimators': 4, 'num_leaves': 4, 'min_child_samples': 12, 'learning_rate': 0.26770501231052046, 'log_max_bin': 7, 'colsample_bytree': 1.0, 'reg_alpha': 0.001348364934537134, 'reg_lambda': 1.4442580148221913}
best roc_auc: 0.9729
training duration:0.01934s


In [19]:
import joblib
 
# Save the model to a file
joblib.dump(best_model, 'best_model.joblib')

StatementMeta(, 8b7e04f3-d0a7-4d41-854c-ccc0ae3c391a, 30, Finished, Available, Finished)

['best_model.joblib']

In [20]:
import pandas as pd
import joblib
 
# Load the trained model
model = joblib.load('best_model.joblib')
 
# Simulated new data with sanitized column names
# simulated_data = [
#     {
#         'Machine_ID': 1, 'Type': 'L', 'Air_temperature_K': 320.5,
#         'Process_temperature_K': 335.5, 'Rotational_speed_rpm': 1700,
#         'Torque_Nm': 50, 'Tool_wear_min': 300, 'Failure_Type': 'No Failure',
#         'Target': 0, 'Timestamp': '2024-11-04 10:00:00'
#     }
# ]

def mloutput(simulated_data): 
    # Convert to DataFrame
    df_simulated = pd.DataFrame(simulated_data)
    
    # Encode 'Type' column
    df_simulated['Type'] = type_encoder.transform(df_simulated[['Type']]).astype(int)
    
    # Select feature columns
    X_simulated = df_simulated[feature_columns]
    
    # Predict probabilities
    probabilities = model.predict_proba(X_simulated)
    
    # Define failure classes
    failure_classes = [
        'No Failure', 'Heat Dissipation Failure', 'Power Failure',
        'Overstrain Failure', 'Tool Wear Failure', 'Random Failures'
    ]
    
    # Create DataFrame with probabilities
    probabilities_df = pd.DataFrame(probabilities, columns=failure_classes)
    
    # Combine with original data
    results_df = pd.concat([df_simulated.reset_index(drop=True), probabilities_df], axis=1)
    
    # Display results
    print(results_df)
    return results_df

StatementMeta(, 8b7e04f3-d0a7-4d41-854c-ccc0ae3c391a, 31, Finished, Available, Finished)

In [18]:
# import mlflow
# from synapse.ml.predict import MLFlowTransformer
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col
# import pandas as pd

# # Initialize Spark session
# spark = SparkSession.builder.appName("MLFlowTransformerExample").getOrCreate()

# # Simulated data
# simulated_data = [
#     {
#         'MachineID': 1, 'Type': 'L', 'Airtemperature': 320.5,
#         'Processtemperature': 335.5, 'Rotationalspeed': 1700,
#         'Torque': 50, 'Toolwear': 300, 'FailureType': 'No Failure',
#         'Target': 0, 'Timestamp': '2024-11-04 10:00:00'
#     }
# ]

# # Create pandas DataFrame
# df_simulated = pd.DataFrame(simulated_data)

# # Encode 'Type' column
# df_simulated['Type'] = type_encoder.transform(df_simulated[['Type']]).astype(int)

# # Convert pandas DataFrame to PySpark DataFrame
# spark_df_simulated = spark.createDataFrame(df_simulated)

# # Define feature columns
# feature_columns = ["Type", "Airtemperature", "Processtemperature", "Rotationalspeed", "Torque", "Toolwear"]

# # Initialize MLFlowTransformer
# model = MLFlowTransformer(
#     inputCols=feature_columns,
#     outputCol="predictions",
#     modelName="AutoML_model_v1",
#     modelVersion=1
# )

# # Predict probabilities
# predictions_df = model.predict_proba(spark_df_simulated)

# # Convert predictions to pandas DataFrame
# predictions_pd_df = predictions_df.toPandas()

# # Define failure classes
# failure_classes = [
#     'No Failure', 'Heat Dissipation Failure', 'Power Failure',
#     'Overstrain Failure', 'Tool Wear Failure', 'Random Failures'
# ]

# # Create DataFrame with probabilities
# probabilities_df = pd.DataFrame(predictions_pd_df["predictions"].tolist(), columns=failure_classes)

# # Combine with original data
# results_df = pd.concat([df_simulated.reset_index(drop=True), probabilities_df], axis=1)

# # Display results
# print(results_df)


StatementMeta(, 8b7e04f3-d0a7-4d41-854c-ccc0ae3c391a, 27, Finished, Available, Finished)

/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalEncoder was fitted without feature names
  warnings.warn(


/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.


AttributeError: 'MLFlowTransformer' object has no attribute 'predict_proba'

StatementMeta(, 8b7e04f3-d0a7-4d41-854c-ccc0ae3c391a, 28, Finished, Available, Finished)

StatementMeta(, 8b7e04f3-d0a7-4d41-854c-ccc0ae3c391a, 29, Finished, Available, Finished)